### Data creation for ENSO events and their vertical depth

Compute ONI values, then extract corresponding tropical Pacific region SSTs for ENSO events. Save as netCDFs for future use. Save El Nino, La Nina, and corresponding experiment climatology (for computing ENSO event anomalies).

**Figure by: Maria J. Molina, NCAR**

In [1]:
import xarray as xr
import numpy as np
from climatico import enso
import matplotlib.pyplot as plt
import cftime
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from climatico.util import weighted_mean, pacific_lon
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from shapely.geometry.polygon import Polygon
from datetime import timedelta

In [2]:
from config import directory_figs, directory_data

In [3]:
def grab_enso_spatial(ds, indices, ds_oni, year1, year2, cutoff, filevar='SST', quarter=12):
    """
    Use the input seasonal file ('QS-DEC') and precomputed Nino-index to extract Nino(a) 
    event indices in time array of SST file.
    Then extract the variable associated with the Nino(a) events. These focus on DJF months.
    
    Args:
        ds (xarray data array): Seasonal variable to filter for plotting 
                                    (e.g., ds.resample(time='QS-DEC').mean(skipna=True)).
        indices (numpy array): ONI values as numpy array 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True).values).
        ds_oni (xarray data array): ONI seasonal xarray 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True)).
        year1 (int): First year for range filtering.
        year2 (int): Second year for range filtering.
        cutoff (float): Cutoff of ENSO events. E.g., +/-0.5 for ONI.
        filevar (str): Variable name for input ds. Defaults to ``SST.``
        quarter (int): Month of quarter start (e.g., 12, 3, 6, 9).
    """
    # do checks
    check1 = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    check2 = ds_oni.isel(time=np.where(indices>=cutoff)[0])[(ds_oni.isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    assert np.all(check1.values==check2.values), "Nino events don't match"
    print("Nino check passed")
    
    check1 = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    check2 = ds_oni.isel(time=np.where(indices<=cutoff)[0])[(ds_oni.isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==quarter)].coords['time']
    assert np.all(check1.values==check2.values), "Nina events don't match"
    print("Nina check passed")
    
    ### climo
    climatology = ds[filevar][(ds[filevar]['time'].dt.month==quarter)]
    climatology = climatology[(climatology['time'].dt.year>=year1)&(climatology['time'].dt.year<=year2)]
    climo = climatology.mean(dim='time')
    
    ### nino
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==quarter)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    # take spatial mean for plotting
    nino = sv_indices.mean(dim='time')
    print("Nino done")

    ### nina
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==quarter)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    # take spatial mean for plotting
    nina = sv_indices.mean(dim='time')
    print("Nina done")
    
    return nino, nina, climo

In [4]:
# list of filenames to do this for:
# SSTs
file_g02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.*.nc'
file_g04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.SST.*.nc'
file_p02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.SST.*.nc'
file_p04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.SST.*.nc'
file_psalt = 'b1d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.*.nc'
file_cntrl = 'b1d.e11.B1850C5CN.f09_g16.005.pop.h.SST.*.nc'
# list of filenames to do this for
TEMP_g02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.TEMP.*.nc'
TEMP_g04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.TEMP.*.nc'
TEMP_p02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.TEMP.*.nc'
TEMP_p04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.TEMP.*.nc'
TEMP_psalt = 'b.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.TEMP.*.nc'
TEMP_cntrl = 'b.e11.B1850C5CN.f09_g16.005.pop.h.TEMP.*.nc'

In [5]:
nino = enso.DefineNino(nino='nino34', lats='lat', lons='lon', cutoff=0.5, runningmean=3)
nino_TEMP = enso.DefineNino(nino='pacslab', cutoff=0.5, runningmean=3)

In [6]:
####################################################################

ds2 = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds2 = ds2.assign_coords(time=ds2.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds2 = ds2.isel(z_t=0).sel(time=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 1, 1, 0, 0)))

print("indices started")
# rolling climo
control_ssts_roll = nino.monthly_climo(ds2['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
control_nino = nino.compute_index(ds2['SST'], control_ssts_roll, 
                                  linear_detrend=False, lat_name='lat')
# grab numpy array
control_index = control_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_cntrl}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)
# fix time coord
ds_ = ds_.assign_coords(time=ds_.coords['time'] - timedelta(days=17))
ds_ = ds_.sel(time=slice(cftime.DatetimeNoLeap(800, 1, 1, 0, 0),cftime.DatetimeNoLeap(1600, 1, 1, 0, 0)))

z_t_coord = ds_['TEMP'].resample(time='AS').mean(skipna=True).mean(dim=['time'],skipna=True).mean(dim=['nlat'],skipna=True).sel(
            z_t=slice(500., 300. * 100.)).coords['z_t'].values

lon_coord = ds_['TEMP'].coords['TLONG'].values

controlnino, controlnina, controlclimo = grab_enso_spatial(
                                              ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500., 300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                              indices = control_index, 
                                              ds_oni = control_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 801, 
                                              year2 = 1599,
                                              cutoff = nino.cutoff, 
                                              filevar = 'TEMP')

####################################################################

indices started
indices done
Nino check passed
Nina check passed
Nino done
Nina done


In [7]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo
g02sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
g02sv_nino = nino.compute_index(ds1['SST'], g02sv_ssts_roll,
                                linear_detrend=False, lat_name='lat')
# grab numpy array
g02sv_index = g02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_g02sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)
# fix time coord
ds_ = ds_.assign_coords(time=ds_.coords['time'] - timedelta(days=17))

g02svnino, g02svnina, g02svclimo = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = g02sv_index, 
                                          ds_oni = g02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started
indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [8]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo
g04sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
g04sv_nino = nino.compute_index(ds1['SST'], g04sv_ssts_roll,
                                linear_detrend=False, lat_name='lat')
# grab numpy array
g04sv_index = g04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_g04sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)
# fix time coord
ds_ = ds_.assign_coords(time=ds_.coords['time'] - timedelta(days=17))

g04svnino, g04svnina, g04svclimo = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = g04sv_index, 
                                          ds_oni = g04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started
indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [9]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo
p02sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
p02sv_nino = nino.compute_index(ds1['SST'], p02sv_ssts_roll,
                                linear_detrend=False, lat_name='lat')
# grab numpy array
p02sv_index = p02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_p02sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)
# fix time coord
ds_ = ds_.assign_coords(time=ds_.coords['time'] - timedelta(days=17))

p02svnino, p02svnina, p02svclimo = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = p02sv_index, 
                                          ds_oni = p02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started
indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [10]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# rolling climo
p04sv_ssts_roll = nino.monthly_climo(ds1['SST'].chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
p04sv_nino = nino.compute_index(ds1['SST'], p04sv_ssts_roll,
                                linear_detrend=False, lat_name='lat')
# grab numpy array
p04sv_index = p04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_p04sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)
# fix time coord
ds_ = ds_.assign_coords(time=ds_.coords['time'] - timedelta(days=17))

p04svnino, p04svnina, p04svclimo = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'],skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = p04sv_index, 
                                          ds_oni = p04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started
indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [11]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                        combine='by_coords',
                        preprocess=nino.nino)
# fix time coord
ds1 = ds1.assign_coords(time=ds1.coords['time'] - timedelta(days=17))
# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

# psalt index computation
ds8 = xr.open_mfdataset(f'{directory_data}b2d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.030101-035012.nc',
                       combine='by_coords',
                       preprocess=nino.nino)
# fix time coord
ds8 = ds8.assign_coords(time=ds8.coords['time'] - timedelta(days=17))
ds8 = ds8.sel(time=slice('0301-01-01 00:00:00', '0351-01-01 00:00:00'))

# attach first 100 years
pnew = xr.concat([ds1['SST'].sel(time=slice('0001-01-01 00:00:00', '0301-01-01 00:00:00')).drop('z_t'),
                  ds8['SST']], dim='time')

print("indices started")
# rolling climo 
psalt_ssts_roll = nino.monthly_climo(pnew.chunk({'time':900}), yrsroll=30, centered=True, time='time')
# compute nino index
psalt_nino = nino.compute_index(pnew, psalt_ssts_roll, 
                                linear_detrend=False, lat_name='lat')
psalt_nino = psalt_nino.sel(time=slice('0001-01-01 00:00:00', '0301-01-01 00:00:00'))
# grab numpy array
psalt_index = psalt_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_psalt}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)
# fix time coord
ds_ = ds_.assign_coords(time=ds_.coords['time'] - timedelta(days=17))

psaltnino, psaltnina, psaltclimo = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = psalt_index, 
                                          ds_oni = psalt_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 101, 
                                          year2 = 250, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started
indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


Nino check passed
Nina check passed
Nino done
Nina done


In [12]:
#%%capture
cntrl_diff_nino = controlnino.values
g02sv_diff_nino = g02svnino.values
g04sv_diff_nino = g04svnino.values
p02sv_diff_nino = p02svnino.values
p04sv_diff_nino = p04svnino.values
psalt_diff_nino = psaltnino.values

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/m

In [13]:
data_assemble=xr.Dataset({
                 'cntrl_nino':(['z_t','a'], cntrl_diff_nino),
                 'g02sv_nino':(['z_t','a'], g02sv_diff_nino),
                 'g04sv_nino':(['z_t','a'], g04sv_diff_nino),
                 'p02sv_nino':(['z_t','a'], p02sv_diff_nino),
                 'p04sv_nino':(['z_t','a'], p04sv_diff_nino),
                 'psalt_nino':(['z_t','a'], psalt_diff_nino),
                 'TLONG':(['nlat','nlon'],  lon_coord)},
                 coords = {'z_t':(['z_t'],  z_t_coord)})

In [14]:
data_assemble.to_netcdf(f'{directory_data}ensoslabs_ninodata.nc')

In [15]:
#%%capture
cntrl_diff_nina = controlnina.values
g02sv_diff_nina = g02svnina.values
g04sv_diff_nina = g04svnina.values
p02sv_diff_nina = p02svnina.values
p04sv_diff_nina = p04svnina.values
psalt_diff_nina = psaltnina.values

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/m

In [16]:
data_assemble=xr.Dataset({
                 'cntrl_nina':(['z_t','a'], cntrl_diff_nina),
                 'g02sv_nina':(['z_t','a'], g02sv_diff_nina),
                 'g04sv_nina':(['z_t','a'], g04sv_diff_nina),
                 'p02sv_nina':(['z_t','a'], p02sv_diff_nina),
                 'p04sv_nina':(['z_t','a'], p04sv_diff_nina),
                 'psalt_nina':(['z_t','a'], psalt_diff_nina),
                 'TLONG':(['nlat','nlon'],  lon_coord)},
                 coords = {'z_t':(['z_t'],  z_t_coord)})

In [17]:
data_assemble.to_netcdf(f'{directory_data}ensoslabs_ninadata.nc')

In [18]:
#%%capture
cntrl_diff_climo = controlclimo.values
g02sv_diff_climo = g02svclimo.values
g04sv_diff_climo = g04svclimo.values
p02sv_diff_climo = p02svclimo.values
p04sv_diff_climo = p04svclimo.values
psalt_diff_climo = psaltclimo.values

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/m

In [19]:
data_assemble=xr.Dataset({
                 'cntrl_climo':(['z_t','a'], cntrl_diff_climo),
                 'g02sv_climo':(['z_t','a'], g02sv_diff_climo),
                 'g04sv_climo':(['z_t','a'], g04sv_diff_climo),
                 'p02sv_climo':(['z_t','a'], p02sv_diff_climo),
                 'p04sv_climo':(['z_t','a'], p04sv_diff_climo),
                 'psalt_climo':(['z_t','a'], psalt_diff_climo),
                 'TLONG':(['nlat','nlon'],  lon_coord)},
                 coords = {'z_t':(['z_t'],  z_t_coord)})

In [20]:
data_assemble.to_netcdf(f'{directory_data}ensoslabs_climodata.nc')